# Compute the prior probability of treatment using permutation 

In [1]:
import itertools
import statistics

import pandas

from hetio.permute import permute_pair_list

In [2]:
from tqdm import tqdm

In [3]:
# Read treatments
treatment_df = pandas.read_table('../summary/indications.tsv')
treatment_df = treatment_df.query("rel_type == 'TREATS_CtD'")
treatment_df.head(2)

,compound_id,compound_name,disease_id,disease_name,rel_type
0,DB01048,Abacavir,DOID:635,acquired immunodeficiency syndrome,TREATS_CtD
1,DB05812,Abiraterone,DOID:10283,prostate cancer,TREATS_CtD


In [4]:
# Create node to degree dictionaries
compound_to_degree = dict(treatment_df.compound_id.value_counts())
disease_to_degree = dict(treatment_df.disease_id.value_counts())

In [5]:
# A degree (compound_degree, disease_degree) to all potential edges with that degree
degree_to_edges = dict()

rows = list()
for (c, c_deg), (d, d_deg) in itertools.product(compound_to_degree.items(), disease_to_degree.items()):
    rows.append((c, d, c_deg, d_deg))
    degree = c_deg, d_deg
    edge = c, d
    degree_to_edges.setdefault(degree, set()).add(edge)

pair_df = pandas.DataFrame(rows, columns=['compound_id', 'disease_id', 'compound_treats', 'disease_treats'])
pair_df = pair_df.sort_values(['compound_id', 'disease_id'])

In [6]:
treatments = list(zip(treatment_df.compound_id, treatment_df.disease_id))

In [7]:
# Burn In
pair_list, stats = permute_pair_list(treatments, multiplier=10)
pandas.DataFrame(stats)

,attempts,complete,cumulative_attempts,duplicate,excluded,same_edge,self_loop,unchanged,undirected_duplicate
0,616,0.100163,615,0.185065,0,0.000000,0,0.230894,0
1,615,0.200163,1230,0.178862,0,0.000000,0,0.126829,0
2,615,0.300163,1845,0.203252,0,0.001626,0,0.089431,0
3,615,0.400163,2460,0.191870,0,0.003252,0,0.076423,0
4,615,0.500163,3075,0.204878,0,0.000000,0,0.110569,0
5,615,0.600163,3690,0.185366,0,0.000000,0,0.105691,0
6,615,0.700163,4305,0.185366,0,0.004878,0,0.091057,0
7,615,0.800163,4920,0.185366,0,0.000000,0,0.097561,0
8,615,0.900163,5535,0.160976,0,0.000000,0,0.089431,0
9,614,1.000000,6149,0.188925,0,0.003257,0,0.097561,0


In [8]:
# Set the multiplier based on the burn in stats
multiplier = 3

In [9]:
# Calculate the number of perms
n_perm = treatment_df.compound_id.nunique() * treatment_df.disease_id.nunique()
n_perm = int(n_perm * 25)
n_perm

616850

In [10]:
%%time

# Initialize a dictionary of degree to empirical probability list
degree_to_probs = {x: list() for x in degree_to_edges}

# Perform n_perm permutations
for i in tqdm(range(n_perm)):
    # Permute
    pair_list, stats = permute_pair_list(pair_list, multiplier=multiplier, seed=i)
    
    # Update
    pair_set = set(pair_list)
    
    # modifies the original degree_to_probs dictionary
    
    for degree, probs in degree_to_probs.items():
        edges = degree_to_edges[degree]
        probs.append(len(edges & pair_set) / len(edges))

100%|██████████| 616850/616850 [1:50:38<00:00, 92.92it/s]

CPU times: user 1h 50min 43s, sys: 9.14 s, total: 1h 50min 52s
Wall time: 1h 50min 38s


In [11]:
%%time
rows = list()
for (c_deg, d_deg), probs in tqdm(degree_to_probs.items()):
    mean = statistics.mean(probs)
    std_error = statistics.stdev(probs) / len(probs) ** 0.5
    rows.append((c_deg, d_deg, mean, std_error))
perm_df = pandas.DataFrame(rows, columns=['compound_treats', 'disease_treats', 'prior_perm', 'prior_perm_stderr'])
perm_df = perm_df.sort_values(['compound_treats', 'disease_treats'])

100%|██████████| 286/286 [21:18<00:00,  4.37s/it]

CPU times: user 21min 18s, sys: 72 ms, total: 21min 19s
Wall time: 21min 18s


In [12]:
# Add unpermuted treatment prevalence columns
rows = list()
treatment_set = set(treatments)
for (c_deg, d_deg), edges in degree_to_edges.items():
    n_treatments = len(edges & treatment_set)
    rows.append((c_deg, d_deg, n_treatments, len(edges)))
degree_prior_df = pandas.DataFrame(rows, columns=['compound_treats', 'disease_treats', 'n_treatments', 'n_possible'])
degree_prior_df = perm_df.merge(degree_prior_df)
degree_prior_df = degree_prior_df.sort_values(['compound_treats', 'disease_treats'])

In [13]:
degree_prior_df.tail(2)

,compound_treats,disease_treats,prior_perm,prior_perm_stderr,n_treatments,n_possible
284,15,44,0.669608,0.000599,0,1
285,15,56,0.730101,0.000565,1,1


In [14]:
degree_prior_df.to_csv('data/degree-prior.tsv', sep='\t', index=False, float_format='%.6g')

In [15]:
obs_prior_df = pair_df.merge(perm_df)

In [16]:
obs_prior_df.head(2)

,compound_id,disease_id,compound_treats,disease_treats,prior_perm,prior_perm_stderr
0,DB00014,DOID:0050741,2,2,0.006101,0.000005
1,DB00014,DOID:1024,2,2,0.006101,0.000005


In [17]:
len(obs_prior_df)

24674

In [18]:
obs_prior_df.to_csv('data/observation-prior.tsv', sep='\t', index=False, float_format='%.6g')